In [ ]:
import os
import pandas as pd
import nltk
import gensim.downloader as api
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from nltk.tokenize.treebank import TreebankWordDetokenizer
import matplotlib.pyplot as plt
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from nltk.corpus import stopwords

In [ ]:
def default_clean(x):
    if type(x) is str:
        x = x.lower()
        x = re.sub('<[^<]+?>', ' ', x)
        x = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', x, flags=re.MULTILINE)
        x = re.sub('[^A-Z a-z]+', ' ', x)
        return x
    else:
        return x

In [ ]:
def removeStopWords(words):
    removed = [w for w in words if not w in stopWords]
    
    return removed;

In [ ]:
def removeSymbols(words):
    removed = [w for w in words if w.isalnum()]
    
    return removed;

In [ ]:
def embedSentence(sentence):
    embed = map(embedding, sentence)
    
    return list(embed);

In [ ]:
def embedding(word):
    try:
        embed = word_vectors[word]
    except Exception:
        embed = word_vectors['unk']
        
    return embed;

In [ ]:
def averageEmbedding(sentence):
    average = np.mean(sentence, axis=0)    
    return average;

In [ ]:
def featurize(sentence, dataframe):
    sentence = pd.Series(sentence, index=FeatureColumns)
    dataframe = dataframe.append(sentence, ignore_index=True)
    
    return dataframe;

In [ ]:
sample = pd.read_csv(r'scored_dataset.csv')

In [ ]:
for i in range(len(removal_list)):
    sample = sample[sample['main_cat'] != removal_list[i]]

In [ ]:
dataDF = sample[['train_feature','main_cat']]

In [ ]:
dataDF['train_feature'] = dataDF['train_feature'].apply(default_clean)

In [ ]:
dataDF['train_feature'] = dataDF['train_feature'].apply(nltk.word_tokenize)

In [ ]:
stopWords = nltk.corpus.stopwords.words('english')

In [ ]:
dataDF['train_feature'] = dataDF['train_feature'].apply(removeStopWords)
dataDF['train_feature'] = dataDF['train_feature'].apply(removeSymbols)

In [ ]:
word_vectors = api.load('glove-wiki-gigaword-300')

In [ ]:
embeddedDF = dataDF.copy()
embeddedDF['train_feature'] = embeddedDF['train_feature'].apply(embedSentence)
embeddedDF = embeddedDF[embeddedDF['train_feature'].astype(str)!='[]']
keepIndices = embeddedDF.index

In [ ]:
embeddedDF['train_feature'] = embeddedDF['train_feature'].apply(averageEmbedding)
embeddedDF.index = list(range(0, len(embeddedDF)))

In [ ]:
FeatureColumns = []
for i in range(0, 300):
    FeatureColumns.append('e'+str(i))
FeatureDF = pd.DataFrame(columns = FeatureColumns)

for i in range(0, len(embeddedDF['train_feature'])):
    FeatureDF = featurize(list(embeddedDF['train_feature'][i]), FeatureDF)

In [ ]:
X = StandardScaler().fit_transform(FeatureDF)

In [ ]:
from sklearn.cluster import KMeans
k_means = KMeans(n_clusters=3)
model = k_means.fit(X)
y_hat = k_means.predict(X)

In [ ]:
sample['clusters'] = y_hat

In [ ]:
sample.reset_index(drop=True, inplace=True)

In [ ]:
sample.to_csv('scored_dataset.csv')

In [ ]:
FeatureDF.to_csv('sentence_embeddings.csv')